In [17]:
# import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib
import os

from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [18]:
# Reading and consultation

df = pd.read_csv(r'./SEER Breast Cancer Dataset .csv', encoding='ascii')
df.head(2)

,Age,Race,Marital Status,Unnamed: 3,T Stage,N Stage,6th Stage,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,43,"Other (American Indian/AK Native, Asian/Pacifi...",Married (including common law),NaN,T2,N3,IIIC,Moderately differentiated; Grade II,Regional,40,Positive,Positive,19,11,1,Alive
1,47,"Other (American Indian/AK Native, Asian/Pacifi...",Married (including common law),NaN,T2,N2,IIIA,Moderately differentiated; Grade II,Regional,45,Positive,Positive,25,9,2,Alive


In [19]:
# Drop unvalide column 'Unnamed'
df = df.drop(columns=['Unnamed: 3'])
df.columns


Index(['Age', 'Race ', 'Marital Status', 'T Stage ', 'N Stage', '6th Stage',
       'Grade', 'A Stage', 'Tumor Size', 'Estrogen Status',
       'Progesterone Status', 'Regional Node Examined',
       'Reginol Node Positive', 'Survival Months', 'Status'],
      dtype='object')

In [20]:
# Nombre de caractéristiques (colonnes) et d'échantillons (lignes)
num_features = df.shape[1]
num_samples = df.shape[0]
print("\nNombre de caractéristiques (colonnes) :", num_features)
print("Nombre d'échantillons (lignes) :", num_samples)


Nombre de caractéristiques (colonnes) : 15
Nombre d'échantillons (lignes) : 7108


In [21]:
# Distribution des statuts (Alive vs Dead)
status_distribution = df['Status'].value_counts()
print("\nDistribution des statuts :")
print(status_distribution)


Distribution des statuts :
Status
Alive     4472
Dead      2634
Status       1
Name: count, dtype: int64


In [34]:
# Spliting the Data into training Set qnd test set
# Conserving data 
dfs = df
# Nettoyage des noms des colonnes
df.columns = df.columns.str.strip()

# Get The target Status ( "Dead or Alive")
df['Status_encoded'] = df['Status'].map({'Alive': 0, 'Dead': 1})
Y = df['Status_encoded']

# Choosing features/inputs
dfs = dfs.drop(columns=['Status'])
X = df[['Age','Marital Status', 'Tumor Size', 'Grade', 'T Stage', 'N Stage','Regional Node Examined',
        'Reginol Node Positive','Survival Months']]
# Train_Set and Test_Set
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print("Inputs :  \n", X_train.columns)
print("Target :  \n", Y_train)


Inputs :  
 Index(['Age', 'Marital Status', 'Tumor Size', 'Grade', 'T Stage', 'N Stage',
       'Regional Node Examined', 'Reginol Node Positive', 'Survival Months'],
      dtype='object')
Target :  
 3606    0.0
794     0.0
3366    0.0
1536    0.0
1561    0.0
       ... 
3772    0.0
5191    1.0
5226    0.0
5390    0.0
860     1.0
Name: Status_encoded, Length: 5686, dtype: float64


In [35]:
# Fitting Models and getting the best one
def train_and_evaluate_models(X_Train, X_Test, Y_Train, Y_Test):
    # Identify numeric and categorical columns
    numeric_features = X_Train.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = X_Train.select_dtypes(include=['object']).columns

    # Preprocessing for numeric data
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])

    # Preprocessing for categorical data
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Combine preprocessors in a column transformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ])

    # Initialize models
    models = {
        'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
        'Decision Tree': DecisionTreeClassifier(random_state=42),
        'Random Forest': RandomForestClassifier(random_state=42),
        'SVM': SVC(random_state=42)
    }

    results = {}
    for model_name, model in models.items():
        # Create a pipeline with preprocessing and model
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ])
        
        # Train the model
        pipeline.fit(X_Train, Y_Train)
        
        # Make predictions
        y_pred = pipeline.predict(X_Test)
        
        # Evaluate model
        accuracy = accuracy_score(Y_Test, y_pred)
        cm = confusion_matrix(Y_Test, y_pred)
        cr = classification_report(Y_Test, y_pred)
        results[model_name] = {
            'model': model,
            'accuracy': accuracy,
            'confusion_matrix': cm,
            'classification_report': cr
        }

        # Plot confusion matrix
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title(f'Confusion Matrix - {model_name}\nAccuracy: {accuracy:.4f}')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.show()

        print(f"\nClassification Report for {model_name}:")
        print(cr)

    return results

# Train and evaluate models
results = train_and_evaluate_models(X_train, X_test, Y_train, Y_test)

# Function to find the best model
def get_best_model(results):
    best_accuracy = 0
    best_model_name = None
    
    for name, result in results.items():
        if result['accuracy'] > best_accuracy:
            best_accuracy = result['accuracy']
            best_model_name = name
    
    return best_model_name, best_accuracy

# Get the best model
best_model_name, best_accuracy = get_best_model(results)
print(f"\nBest Model: {best_model_name} with Accuracy: {best_accuracy:.4f}")

ValueError: Input y contains NaN.